In [12]:
import pandas as pd

passed_data = []
data_procession = []
with open('/home/ouconstand/data/Virus_Host/Virion.csv','r', encoding='ISO-8859-1') as f:
    for line in f.read().splitlines():
        data = line.split('\t')
        if len(data) < 34:
            passed_data.append(data)
        else:
            if data[8] == '' or data[13] == '' or data[3] == '': # delete non_family information data
                pass
            else:
                data_procession.append(data)

data = pd.DataFrame(data_procession[1:-1],columns=data_procession[0])

In [13]:
with open('/home/ouconstand/ICTV_taxid.txt','r') as f:
    ICTV = [i.strip('\n') for i in f.readlines()]

In [14]:
with open('/home/ouconstand/wang_0207.txt','r') as f:
    VFamily = [i.strip('\n') for i in f.readlines()]

In [15]:
print(data.shape)
virion_ictv = data[data['VirusTaxID'].isin(ICTV)]
print(virion_ictv.shape)
even_screen = virion_ictv[virion_ictv['VirusFamily'].isin(VFamily)]
print(even_screen.shape)

(478970, 34)
(443620, 34)
(346735, 34)


In [16]:
len(set(even_screen['VirusTaxID']))

1531

In [19]:
multi_infect_human = pd.DataFrame(columns=['VirusFamily','Infect_MultiFamily_Except_hominidae','IMFE_Infect_human','One_Family','OF_Infect_human'])

virus_family = VFamily

n = 0
for one_family in virus_family:
    
    train_data = even_screen[even_screen['VirusFamily'] == one_family]

    virusID_hostfamily = train_data[['VirusTaxID','HostFamily']]
    virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
    virusID_hostfamily.reset_index(drop=True) # reset index

    dic = {}

    mt_except_human = 0
    mte_IH = 0
    one_f = 0
    of_H = 0

    for i,j in zip(virusID_hostfamily['VirusTaxID'], virusID_hostfamily['HostFamily']):
        dic[i] = dic.get(i, []) + [j]

    for value in dic.values():
        if len(value) >= 3:
            if 'hominidae' in value:
                mt_except_human += 1
                mte_IH += 1
            else:
                mt_except_human += 1
        if len(value) == 2:
            if 'hominidae' in value:
                pass
            else:
                mt_except_human += 1
        if len(value) == 1:
            one_f += 1
            if 'hominidae' in value:
                of_H += 1
            
    multi_infect_human.loc[n] = [one_family.capitalize(), mt_except_human, mte_IH, one_f,of_H]
    n += 1

<ipython-input-19-50587057a065>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-19-50587057a065>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  virusID_hostfamily.drop_duplicates(subset=['VirusTaxID','HostFamily'],inplace=True) # drop duplication
<ipython-input-19-50587057a065>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [20]:
multi_infect_human

,VirusFamily,Infect_MultiFamily_Except_hominidae,IMFE_Infect_human,One_Family,OF_Infect_human
0,Adenoviridae,24,3,60,4
1,Anelloviridae,11,7,86,50
2,Arenaviridae,13,7,37,3
3,Astroviridae,12,4,7,1
4,Bornaviridae,5,1,5,0
5,Caliciviridae,6,3,6,0
6,Circoviridae,14,3,70,16
7,Coronaviridae,23,6,20,2
8,Filoviridae,6,6,5,0
9,Flaviviridae,50,37,33,4


In [21]:
multi_infect_human.to_csv('/mnt/c/Users/ouyangkang/Desktop/multi_infect_to_human.csv',index=None)